# Projet de session - IFT712

## Configuration

In [15]:
pip install -r '../requirements.txt'

Note: you may need to restart the kernel to use updated packages.


In [1]:
import data_manager.data_manager
import data_manager.data_organizer

In [2]:
# Load the database.
db = data_manager.data_manager.DatabaseManager('')
db.load()
# Get the datasets.
dataset = db.get_train_dataset().drop('id', axis=1)

In [3]:
do = data_manager.data_organizer.DataOrganizer()
do.shuffle(dataset, dataset.columns)
do.standardize(dataset, dataset.columns[1:])
do.numerize(dataset, dataset.columns[0])

In [4]:
# Split the dataset in 70%/30%.
training_dataset = dataset[:int(dataset.shape[0] * 0.7)]
test_dataset = dataset[int(dataset.shape[0] * 0.7):]

# Separate the inputs from the targets.
training_inputs = training_dataset.drop('species', axis=1)
training_targets = training_dataset['species']

test_inputs = test_dataset.drop('species', axis=1)
test_targets = test_dataset['species']

In [5]:
X = dataset.drop('species', axis=1)
y = dataset['species']

## KNN :

In [ ]:
import models.kneighbors_classifier

clf=models.kneighbors_classifier.KneighborsClassifier()

In [ ]:
clf.train(training_inputs, training_targets)

In [ ]:
clf.score(test_inputs, test_targets)

## SVM :

In [ ]:
import models.svm_classifier
clf=models.svm_classifier.SVMClassifier()

In [ ]:
clf.train(training_inputs, training_targets)

In [ ]:
clf.score(test_inputs, test_targets)

## Logistic Regression :

In [6]:
import models.logistic_regression_classifier
lr = models.logistic_regression_classifier.LogisticRegressionClassifier()

In [7]:
lr.train(training_inputs, training_targets)

In [8]:
lr.score(test_inputs, test_targets)

0.98989898989899

In [11]:
from sklearn.model_selection import cross_val_score
scores = lr.cross_v(lr, X, y, 5)

TypeError: cross_v() takes 4 positional arguments but 5 were given

## MLP :

In [ ]:
import models.mlp_classifier
mlp = models.mlp_classifier.MLPClassifier()

In [ ]:
mlp.train(training_inputs, training_targets)

In [ ]:
mlp.score(test_inputs, test_targets)

## Perceptron :

In [ ]:
import models.perceptron_classifier
per = models.perceptron_classifier.PerceptronClassifier()

In [ ]:
per.train(training_inputs, training_targets)

In [ ]:
per.score(test_inputs, test_targets)

## SVC :

In [ ]:
import models.svc_classifier
svc = models.svc_classifier.SVCClassifier()

In [ ]:
svc.train(training_inputs, training_targets)

In [ ]:
svc.score(test_inputs, test_targets)